# Func for gigachat API

In [12]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    rq_uid = str(uuid.uuid4())

    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }


    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1
    

def get_chat_completion(auth_token, user_message):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 10,
        "repetition_penalty": 1, 
        "update_interval": 0
    })

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Произошла ошибка: {str(e)}")
        return -1


In [13]:
your_api_token = 'M2M2MDQzOWUtNGJmMy00YzJhLWI4M2MtOGFjNzUzODg4OTEwOmMwMWIxMWM1LTkxMjUtNDc3ZS1hOWQ0LTZlNDk3MDMwZmY2Mw=='

In [14]:
response = get_token(your_api_token)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.eOwaRgyb2UKxho4mGVvxA5BcRFAB5-swD67RnLUCwbpZdwE_qEHXj5Oqw_nyD52EPGBcj5pAltNkBXYTx5ntl_D3QlTg6OoAEwL2HnW7ov0K6friZNB3iMCjN13S1VvHMjeXyfxFY_2gc-RJJ4Fv3aHpWSKinipOx6X4sxyuzkvOCSpR_cP0G7uT1yburY-JorCgYQnpkP9XQsWVGuzGq3mdu7HyOr58JQeKNfdpge1LKYOiwGhZM2g1RVXNKEIMBRshMXoBOIjQEwZN24WcCdZIfGGrFv7xA_cV84QOaBjOkZDTjO0-rGtCdrvJ3CW_QORxeJsJ8R8Kj04aQ7tWnw.v7K7XGsGJ4UVHGzeAaUpXQ.uFp0sXLxkNcSgdJ2HL5XY3YrDPp0fvJGqkAWL0tBAzZvAyMRiNdAxt-cSo8w0tJpW1XPNmwNFSP25E8TNLu2DrrXImAe3mVILR0Yr37jzms-s5n-zLWc-XB1GdpXH4F4IDwGmxpTfuHEiHYLcpJPG23QhFkjWX8mhRcHPSOQQ4sOnCpM0zTau-D-m6lEsw-kcMNpR9vd3uVJ9K80LsmyBMiBTpKqUK05aW_I8uFNgFoEvN7tfODHFYukxVHnohjxdWBcvniXAVtmg1bwPADVu_tQo1us7T1DxTOYbbKY0BQ75Z3STLhNXYLkL3vqacZgJFFhLw9ShLc7jYh_6Z9Kw9H5WcI35f1D5YuNkQTuXQzFlDepd7Ur1aCoOCcNHnunmWlQ1EWKt_KMSdvEEgFnU-zONoofZ_vbb_mCD1DFbeHh1boVtzCa1ylVpthVE2vWQU3aKAbwsFHF-FxPArwgk3fixpqnwnSbor5BUZ4J7TWvGkBUirp3m4BbY-mDpQF8kkfvpbqQbTmIRCaSoVW43yN6c8QVF

In [15]:
prompt = """Ты – эксперт по разметке новостного контента. Твоя задача – определить, к какой одной категории относится данная новость. Ниже приведён список доступных категорий:
0 - Политика: новости о правительстве, выборах, международных отношениях, законодательстве.
1 - Экономика и бизнес: финансовые рынки, корпоративные новости, стартапы, экономические обзоры.
2 - Спорт: результаты матчей, анонсы спортивных событий, интервью, аналитика спортивных событий.
3 - Культура и развлечения: кино, музыка, театр, выставки, обзоры культурных событий.
4 - Технологии и наука: новости IT-сферы, инновации, открытия, гаджеты, исследования.
5 - Здоровье и медицина: новости о медицине, исследования, советы по здоровью, эпидемии.
6 - Общество: социальные тренды, образование, городская жизнь, экология, права человека.
7 - Происшествия и криминал: сообщения о происшествиях, расследованиях, криминальных событиях.
8 - Локальные новости: события, касающиеся определённых регионов или городов.

Инструкция:
Прочитай текст новости, представленный ниже.
Определи основную тему новости, сопоставив её с описаниями категорий.
Выведи в ответ только один класс, соответствующий новости.
Ответ должен состоять из одного слова – названия категории, которое следует после двоеточия.
Например, если новость касается выборов, твой ответ должен быть:
"Политика"

Текст для анализа: /"{text}/". Ответь одни классом из ["Политика","Экономика и бизнес","Спорт","Культура и развлечения","Технологии и наука","Здоровье и медицина","Общество","Происшествия и криминал","Локальные новости"]:"""

In [16]:
text = 'Чиновники, депутаты и правозащитники вернулись к обсуждению «сухого закона» для водителей. Противники возвращения промилле считают, что в России из-за ментальности населения разрешать пить за рулем даже в минимальных дозах нельзя. С тех пор как обнулили допустимую норму, число «пьяных» ДТП только выросло, а водителей с мизерным опьянением повсеместно лишают прав и вымогают крупные взятки.'
prompt.format(text=text)

'Ты – эксперт по разметке новостного контента. Твоя задача – определить, к какой одной категории относится данная новость. Ниже приведён список доступных категорий:\n0 - Политика: новости о правительстве, выборах, международных отношениях, законодательстве.\n1 - Экономика и бизнес: финансовые рынки, корпоративные новости, стартапы, экономические обзоры.\n2 - Спорт: результаты матчей, анонсы спортивных событий, интервью, аналитика спортивных событий.\n3 - Культура и развлечения: кино, музыка, театр, выставки, обзоры культурных событий.\n4 - Технологии и наука: новости IT-сферы, инновации, открытия, гаджеты, исследования.\n5 - Здоровье и медицина: новости о медицине, исследования, советы по здоровью, эпидемии.\n6 - Общество: социальные тренды, образование, городская жизнь, экология, права человека.\n7 - Происшествия и криминал: сообщения о происшествиях, расследованиях, криминальных событиях.\n8 - Локальные новости: события, касающиеся определённых регионов или городов.\n\nИнструкция:\nП

In [17]:
import pandas as pd
import json
import csv
import os

# 读取 CSV 文件
df = pd.read_csv('lenta_news.csv')

# 目标 CSV 文件
output_file = 'lenta_news_with_answers.csv'

# 检查文件是否已存在
write_header = not os.path.exists(output_file)

# 以追加模式写入 CSV 文件
with open(output_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # 写入表头（如果是新文件）
    if write_header:
        writer.writerow(list(df.columns) + ['Ответ'])  # 保留原有列，并添加新列 'Ответ'
    
    # 遍历数据并调用 API 获取答案
    for index, row in df.iterrows():
        try:
            # 调用 API 获取答案
            answer = get_chat_completion(giga_token, prompt.format(text=row['Текст']))
            
            # 从响应中提取内容
            answer_content = answer.json()['choices'][0]['message']['content']
            
            # 写入文件，保留原始数据并添加答案
            writer.writerow(list(row) + [answer_content])
            
            print(f"Processed: {row['Заголовок'][:30]}...")  # 显示进度
        except Exception as e:
            print(f"Error processing: {row['Заголовок'][:30]}..., Error: {str(e)}")


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Processed: Карлсон назвал Украину не суве...


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Processed: На Украине назвали условие для...


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Processed: В российском городе прозвучали...


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Processed: В Диснейленде вспыхнул пожар...


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Processed: При атаке ВСУ на российский ре...


# Markup datasets from transforrmers

In [4]:
import csv
from tqdm import tqdm
from datasets import load_dataset

name = 'IlyaGusev/gazeta' # the news dataset
dataset = load_dataset(name)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
})

In [ ]:
prompt = """Ты – эксперт по разметке новостного контента. Твоя задача – определить, к какой одной категории относится данная новость. Ниже приведён список доступных категорий:
0 - Политика: новости о правительстве, выборах, международных отношениях, законодательстве.
1 - Экономика и бизнес: финансовые рынки, корпоративные новости, стартапы, экономические обзоры.
2 - Спорт: результаты матчей, анонсы спортивных событий, интервью, аналитика спортивных событий.
3 - Культура и развлечения: кино, музыка, театр, выставки, обзоры культурных событий.
4 - Технологии и наука: новости IT-сферы, инновации, открытия, гаджеты, исследования.
5 - Здоровье и медицина: новости о медицине, исследования, советы по здоровью, эпидемии.
6 - Общество: социальные тренды, образование, городская жизнь, экология, права человека.
7 - Происшествия и криминал: сообщения о происшествиях, расследованиях, криминальных событиях.
8 - Локальные новости: события, касающиеся определённых регионов или городов.

Инструкция:
Прочитай текст новости, представленный ниже.
Определи основную тему новости, сопоставив её с описаниями категорий.
Выведи в ответ только один класс, соответствующий новости.
Ответ должен состоять из одного слова – названия категории, которое следует после двоеточия.
Например, если новость касается выборов, твой ответ должен быть:
"Политика"

Текст для анализа: "{text}". Ответь одни классом из ["Политика","Экономика и бизнес","Спорт","Культура и развлечения","Технологии и наука","Здоровье и медицина","Общество","Происшествия и криминал","Локальные новости"]:"""

### Markup only validation set becaus of free token limits

In [ ]:
with open('output2.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['text', 'output'])
    
    for i, data in enumerate(tqdm(dataset['validation'], total=len(dataset['validation']))):
        answer = get_chat_completion(giga_token,prompt.format(text=data['summary'])).json()['choices'][0]['message']['content']
        row = [data['summary'], str(answer)]
        writer.writerow(row)
